# Gallery Example: M/E₂/1 Queue (Erlang Service)

This example demonstrates an M/E₂/1 queueing system:
- **Arrivals**: Poisson process (Exponential inter-arrival times)
- **Service**: Erlang-2 service times (less variable than exponential)
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

The Erlang-2 service distribution has lower variance than exponential, representing more consistent service times.

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_merl1():    """Create M/E₂/1 queueing model"""    model = Network('M/E/1')        # Block 1: nodes    source = Source(model, 'mySource')    queue = Queue(model, 'myQueue', SchedStrategy.FCFS)    sink = Sink(model, 'mySink')        # Block 2: classes    oclass = OpenClass(model, 'myClass')    # Exponential arrivals with rate λ=1    source.set_arrival(oclass, Exp(1))    # Erlang-2 service with mean=0.5 and order=2    queue.set_service(oclass, Erlang.fit_mean_and_order(0.5, 2))        # Block 3: topology    P = model.init_routing_matrix()    P.add_route(oclass, source, queue, 1.0)    P.add_route(oclass, queue, sink, 1.0)    model.link(P)        return model, source, queue, sink, oclass# Create the modelmodel, source, queue, sink, oclass = gallery_merl1()

## Theoretical Analysis for M/E₂/1

For M/E₂/1 with:
- **Arrival rate**: λ = 1 (Exponential with mean=1)
- **Service time**: Erlang-2 with mean=0.5, order=2
- **Utilization**: ρ = λ × E[S] = 1 × 0.5 = 0.5

The Erlang-2 service distribution characteristics:
- **Mean**: 0.5
- **Variance**: (0.5)²/2 = 0.125 (lower than exponential variance = 0.25)
- **Coefficient of Variation**: C²ₛ = 1/2 = 0.5

Lower service variability typically leads to better performance than M/M/1.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)

In [ ]:
# Compare M/E₂/1 with M/M/1
print("\n=== Comparison with M/M/1 ===")

# Create equivalent M/M/1 model
def create_mm1_equivalent():
    model_mm1 = Network('M/M/1-Equivalent')
    source = Source(model_mm1, 'Source')
    queue = Queue(model_mm1, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_mm1, 'Sink')
    
    oclass = OpenClass(model_mm1, 'Class')
    source.set_arrival(oclass, Exp(1))  # Same arrival rate
    queue.set_service(oclass, Exp(2))   # Exponential with mean = 0.5
    
    P = model_mm1.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_mm1.link(P)
    
    return model_mm1

model_mm1 = create_mm1_equivalent()
solver_merl = MVA(model)
solver_mm1 = MVA(model_mm1)

avg_table_merl = solver_merl.avg_table()
avg_table_mm1 = solver_mm1.avg_table()

print("M/E₂/1 Results:")
print(avg_table_merl)

print("\nM/M/1 Results:")
print(avg_table_mm1)

# Extract key metrics for comparison
merl_resp = float(avg_table_merl.iloc[1, 2])  # Queue response time
merl_length = float(avg_table_merl.iloc[1, 3])  # Queue length

mm1_resp = float(avg_table_mm1.iloc[1, 2])  # Queue response time
mm1_length = float(avg_table_mm1.iloc[1, 3])  # Queue length

print(f"\nPerformance Comparison:")
print(f"Response Time: M/E₂/1={merl_resp:.4f}, M/M/1={mm1_resp:.4f}")
print(f"Queue Length: M/E₂/1={merl_length:.4f}, M/M/1={mm1_length:.4f}")
improvement = ((mm1_resp - merl_resp) / mm1_resp * 100) if mm1_resp > merl_resp else ((merl_resp - mm1_resp) / mm1_resp * 100)
print(f"Improvement: {improvement:.1f}% {'better' if mm1_resp > merl_resp else 'worse'} response time with Erlang service")

In [ ]:
# Analyze impact of Erlang order for service times
print("\n=== Impact of Service Time Variability ===")

def create_service_erlang_model(order):
    """Create M/Eₖ/1 model with specified Erlang order for service"""
    model_erl = Network(f'M/E{order}/1')
    source = Source(model_erl, 'Source')
    queue = Queue(model_erl, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_erl, 'Sink')
    
    oclass = OpenClass(model_erl, 'Class')
    source.set_arrival(oclass, Exp(1))
    queue.set_service(oclass, Erlang.fit_mean_and_order(0.5, order))
    
    P = model_erl.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_erl.link(P)
    
    return model_erl

# Test different Erlang orders for service
orders = [1, 2, 3, 5, 10]  # Order 1 = Exponential

print("Service Order | C²ₛ  | Response Time | Queue Length")
print("-" * 50)

for order in orders:
    model_order = create_service_erlang_model(order)
    solver = MVA(model_order)
    avg_table = solver.avg_table()
    
    resp_time = float(avg_table.iloc[1, 2])
    queue_length = float(avg_table.iloc[1, 3])
    cv_squared = 1.0 / order  # Coefficient of variation squared for Erlang
    
    print(f"     {order:2d}       | {cv_squared:.3f} |     {resp_time:.4f}    |    {queue_length:.4f}")

print("\nNote: Higher order (lower C²ₛ) leads to better performance due to reduced service variability.")
print("As order→∞, the system approaches M/D/1 (deterministic service).")

In [ ]:
# Compare with M/H₂/1 (hyperexponential service)
print("\n=== Comparison: Low vs High Service Variability ===")

def create_mhyp1_model():
    """Create M/H₂/1 model with hyperexponential service"""
    model_hyp = Network('M/H2/1')
    source = Source(model_hyp, 'Source')
    queue = Queue(model_hyp, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_hyp, 'Sink')
    
    oclass = OpenClass(model_hyp, 'Class')
    source.set_arrival(oclass, Exp(1))
    queue.set_service(oclass, HyperExp.fit_mean_and_scv(0.5, 4))  # High variability
    
    P = model_hyp.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_hyp.link(P)
    
    return model_hyp

model_hyp = create_mhyp1_model()
model_erl = create_service_erlang_model(5)  # Low variability

solver_hyp = MVA(model_hyp)
solver_erl = MVA(model_erl)

avg_table_hyp = solver_hyp.avg_table()
avg_table_erl = solver_erl.avg_table()

hyp_resp = float(avg_table_hyp.iloc[1, 2])
erl_resp = float(avg_table_erl.iloc[1, 2])

print(f"Service Distribution | C²ₛ | Response Time")
print("-" * 40)
print(f"Erlang-5 (low var)   | 0.2 |    {erl_resp:.4f}")
print(f"Exponential          | 1.0 |    {mm1_resp:.4f}")
print(f"Hyperexp (high var)  | 4.0 |    {hyp_resp:.4f}")

print(f"\nVariability Impact: {(hyp_resp / erl_resp):.1f}x difference between high and low service variability")
print("This demonstrates the fundamental principle: variability degrades queueing performance.")